In [1]:
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed,Add
from tensorflow.keras.models import Model

Using TensorFlow backend.


In [2]:
df_train=pd.read_csv('Bengali_news/train.csv')
df_test=pd.read_csv('Bengali_news/valid.csv')

In [6]:
x_train,y_train,x_test,y_test=df_train.article,df_train.label,df_test.article,df_test.label

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

In [9]:
text_cl_nv=Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())])
text_cl_svm=Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [10]:
text_cl_nv.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [11]:
prediction=text_cl_nv.predict(x_test)

In [12]:
from sklearn.metrics import accuracy_score

In [13]:
accuracy_score(y_test,prediction)

0.4484463276836158

In [15]:
text_cl_svm.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [16]:
pred=text_cl_svm.predict(x_test)

In [17]:
accuracy_score(y_test,pred)

0.8213276836158192

In [34]:
df_test.head()

,title,article,label
0,মাঠে নেমেই অর্ধশতক! রাহুল জানালেন তাঁর সাফল্যে...,গলেরই পুনরাবৃত্তি হয়তো কলম্বোয় ঘটতে চলেছে। শ্...,sports
1,কুমার সম্ভব?,তাহলে কি ঘাটা-আঘাটায় পুলিশের মারধর খেয়ে নীরবে...,state
2,স্বামী-স্ত্রীর ঘনিষ্ঠ মুহূর্তের ছবি সোশ্যাল মি...,পণের দাবিতে নজিরবিহীন ব্ল্যাকমেল স্ত্রীকে। স্ব...,kolkata
3,‘এবেলা’ খবরের প্রতিক্রিয়া: অভীকের সঙ্গে কথা হল...,বাংলার প্যারালিম্পিক সংস্থার সঙ্গে যোগাযোগ হল...,sports
4,‘অ্যায় দিল হ্যায় মুশকিল’-এই অভিনেত্রী কি অন্তঃ...,"‘বোল্ড’, তথাকথিত পারিবারিক ছবির গণ্ডী থেকে বে...",entertainment


In [36]:
df_test.article[1]

' তাহলে কি ঘাটা-আঘাটায় পুলিশের মারধর খেয়ে নীরবে নিভৃতে ফিরে আসার দিন শেষ হতে চলেছে? তাহলে কি পুলিশ ‘দুষ্টের দমন এবং শিষ্টের পালন’ করবে এবার? তাহলে কি পুলিশ এবার পাড়ায় পাড়ায় ফুটবল খেলা বা গাছ লাগানো ছেড়ে পুলিশসুলভ আচরণ করবে?\nদায়িত্ব নেওয়ার পর কলকাতা পুলিশের কমিশনার রাজীব কুমার তাঁর প্রথম ক্রাইম কনফারেন্সে বুধবার যা বলেছেন, তার প্রেক্ষিতেই এইসব প্রশ্ন তৈরি হয়েছে পুলিশমহলে। কারণ, নতুন কমিশনার তাঁর বাহিনীকে ‘সম্মান’ নিয়ে কাজ করার বার্তা দিয়েছেন। বলেছেন, লক্ষ্য রাখতে হবে, যাতে সাধারণ মানুষের পুলিশের উপর আস্থা থাকে। \xa0\r\nরাজীব কুমারের ওই বক্তব্যে প্রকারান্তরে এটাও স্পষ্ট যে, পুলিশ ‘সম্মান’ নিয়ে কাজ করছে না এবং তাদের উপর সাধারণ মানুষের আস্থাও নেই।\xa0\nনতুন কমিশনারের এই বক্তব্য বিগত জমানার প্রতি কোনও ইঙ্গিত কি না, তা নিয়ে বিতর্ক থাকতে পারে। যা নিয়ে তর্ক নেই, নতুন কমিশনার তাঁর বাহিনীর মনোবল চাঙ্গা করতে চাইছেন। রাজ্যের বিভিন্ন প্রান্তের মতো কলকাতা \xa0শহরেও বারবার আক্রান্ত হয়েছে পুলিশ। কিন্তু সেসব ক্ষেত্রে অভিযুক্তদের ধরার ক্ষেত্রে পুলিশি ‘নিষ্ক্রিয়তা’র অভিযোগ উঠেছে বারবার। পুলিশের একাংশ

In [37]:
b='তাহলে কি ঘাটা-আঘাটায় পুলিশের মারধর খেয়ে নীরবে নিভৃতে ফিরে আসার দিন শেষ হতে চলেছে? তাহলে কি পুলিশ ‘দুষ্টের দমন এবং শিষ্টের পালন’ করবে এবার? তাহলে কি পুলিশ এবার পাড়ায় পাড়ায় ফুটবল খেলা বা গাছ লাগানো ছেড়ে পুলিশসুলভ আচরণ করবে?\nদায়িত্ব নেওয়ার পর কলকাতা পুলিশের কমিশনার রাজীব কুমার তাঁর প্রথম ক্রাইম কনফারেন্সে বুধবার যা বলেছেন, তার প্রেক্ষিতেই এইসব প্রশ্ন তৈরি হয়েছে পুলিশমহলে। কারণ, নতুন কমিশনার তাঁর বাহিনীকে ‘সম্মান’ নিয়ে কাজ করার বার্তা দিয়েছেন। বলেছেন, লক্ষ্য রাখতে হবে, যাতে সাধারণ মানুষের পুলিশের উপর আস্থা থাকে। \xa0\r\nরাজীব কুমারের ওই বক্তব্যে প্রকারান্তরে এটাও স্পষ্ট যে, পুলিশ ‘সম্মান’ নিয়ে কাজ করছে না এবং তাদের উপর সাধারণ মানুষের আস্থাও নেই।\xa0\nনতুন কমিশনারের এই বক্তব্য বিগত জমানার প্রতি কোনও ইঙ্গিত কি না, তা নিয়ে বিতর্ক থাকতে পারে। যা নিয়ে তর্ক নেই, নতুন কমিশনার তাঁর বাহিনীর মনোবল চাঙ্গা করতে চাইছেন। রাজ্যের বিভিন্ন প্রান্তের মতো কলকাতা \xa0শহরেও বারবার আক্রান্ত হয়েছে পুলিশ। কিন্তু সেসব ক্ষেত্রে অভিযুক্তদের ধরার ক্ষেত্রে পুলিশি ‘নিষ্ক্রিয়তা’র অভিযোগ উঠেছে বারবার। পুলিশের একাংশের মতে, ওই ‘নিষ্ক্রিয়তা’র জন্যই পুলিশ-প্রশাসনের প্রতি সাধারণ মানুষের আস্থা তলানিতে গিয়ে ঠেকেছে। সে কারণে শুধু রাজনৈতিক দলের নেতা-কর্মীরাই নয়, পুলিশের গায়ে হাত তুলতে কসুর করছে না আমজনতাও। একের পর এক পুলিশ আক্রান্ত হওয়ার ঘটনায় অনেক সময়েই সোশ্যাল নেটওয়ার্কিং সাইটে সরব হতে দেখা গিয়েছে নিচুতলার পুলিশকর্মীদের।\xa0\nসেই প্রেক্ষিতেই নতুন কমিশনারের এদিনের বার্তা। আইপিএস মহলের একাংশের মতে, পুলিশবাহিনী তাদের ‘সম্মান’ নিয়ে কাজ করতে পারলেই সাধারণ মানুষের ‘আস্থা’ যে পুলিশের উপর ফিরবে, তা বিলক্ষণ জানেন রাজীব কুমার। তাই দায়িত্ব নিয়ে ‘সম্মান ও আস্থা’র দু’টি অস্ত্রে শান দিতে চাইছেন তিনি।\xa0\n\nরাজীব কুমার—ফাইল চিত্র\xa0\nএকইসঙ্গে বাহিনীর সদস্যদের অন্যান্য পরিকাঠামোগত সুযোগসুবিধার প্রতিও নজর দিতে শুরু করেছেন রাজীব কুমার। সম্প্রতি লালবাজারের কন্ট্রোল রুমে গিয়েছিলেন তিনি। সেখানকার শৌচালয়টি দেখে সেটিকে নতুন করে তৈরি করার নির্দেশ দিয়েছেন কমিশনার। বলেছেন, কোনও অসুবিধা হলে পুলিশকর্মীরা যেন সরাসরি তাঁর সঙ্গে যোগাযোগ করেন। পুলিশকর্মীদের পরিছন্ন পোশাক পরে ডিউটি করার কথাও বলেছেন রাজীব কুমার। তাঁর বক্তব্য, কোনও থানার কোনও কনস্টেবল অপরিছন্ন উর্দি পরে ডিউটি করলে তা শুধু সেই থানাই নয়, গোটা কলকাতা পুলিশের পক্ষেই ‘অসম্মানজনক’। অভিজ্ঞ অফিসারদের ধারণা, বাহিনীর পরিকাঠামোগত সুযোগসুবিধার প্রতি নজর দিয়ে রাজীব কুমার আসলে বাহিনীতে জনপ্রিয় সৌমেন মিত্রের জুতোয় পা গলাতে চাইছেন। রাজীব কুমার কমিশনার হওয়ায় তাঁর থেকে এক ব্যাচ সিনিয়র সৌমেনকে কলকাতা পুলিশ থেকে রাজ্য পুলিশে সরানো হয়েছে। যা নিয়ে কলকাতা পুলিশের একাংশে মৃদু ক্ষোভও সঞ্চারিত হয়েছিল। বাহিনীর সদস্যদের জন্য বিভিন্ন পরিকাঠামোগত সুবিধা তৈরি করে সৌমেন জনপ্রিয়তা অর্জন করেছিলেন। কলকাতা পুলিশে তাঁর ‘ভক্তে’র সংখ্যাও কম ছিল না। দায়িত্ব নিয়ে রাজীব কুমার সৌমেনের বইয়ের কিছু পাতা ওল্টানোর চেষ্টা শুরু করেছেন বলেই মনে করছেন অফিসারদের একাংশ।\xa0\nএদিন পুলিশ ট্রেনিং স্কুলে ক্রাইম কনফারেন্সে উপস্থিত ছিলেন লালবাজারের শীর্ষকর্তারা। ছিলেন ৬৯টি থানার ওসি (এদিনই লেক থানাকে ভেঙে নতুন রবীন্দ্র সরোবর থানা তৈরির কথা ঘোষণা করেছেন মুখ্যমন্ত্রী), বিভাগীয় ডিসি, এসি এবং গোয়েন্দাবিভাগের অফিসারেরা। ঘণ্টাখানেকের বৈঠকে পুলিশ-নিগ্রহের ঘটনা নিয়ে মুখ খোলেন কমিশনার। বৈঠকে তিনি স্পষ্টই বুঝিয়ে দেন যে, পুলিশের আক্রান্ত হওয়ার ঘটনা কোনওভাবেই বরদাস্ত করা হবে না। ওই ধরনের কোনও ঘটনা ঘটলে ‘কড়াহাতে’ তা মোকাবিলা করতে হবে এবং অভিযুক্তদের শাস্তি নিশ্চিত করতে হবে। এমনকী, কোনও ‘সিভিক পুলিশ’ আক্রান্ত হলেও দ্রুত অভিযোগ দায়ের করে ব্যবস্থা নিতে হবে। যাতে অভিযুক্তের পুলিশ হেফাজত বা জেল হেফাজত নিশ্চিত করা যায়।\xa0\nপুলিশের একটি মহলের অবশ্য এ নিয়ে সংশয় রয়েছে। তাদের বক্তব্য, বেশিরভাগ সময় শাসকদলের নেতাকর্মীদের হাতেই পুলিশ আক্রান্ত হয়। সেক্ষেত্রেও এই নির্দেশ কতটা কার্যকরী হয় সেটা দেখার।\xa0\r\nবৈঠকে নতুন কমিশনার সম্প্রতি গড়িয়াহাটে তৃণমূল যুবনেতার গাড়ির ধাক্কায় পথচারীর মৃত্যুর প্রসঙ্গ তোলেন। তিনি জানান, ট্র্যাফিক ও স্থানীয় থানার মধ্যে সমন্বয় রাখতে হবে। কোনও দুর্ঘটনা ঘটলে তার প্রাথমিক তদন্ত করবে স্থানীয় থানা। পরে ঘটনার গুরুত্ব বুঝে ট্র্যাফিক বিভাগ তদন্ত করবে। পাশাপাশিই তাঁর পরামর্শ, কোনও ঘটনায় জেনারেল ডায়েরি নেওয়ার সময় অভিযোগকারীর নামের সঙ্গে ফোন নম্বরও লিখে রাখা হোক। যাতে প্রয়োজনে তাঁর সঙ্গে পরে যোগাযোগ করা যেতে পারে।\xa0\nডাকাতি দমন এবং মোটরগাড়ি চুরি দমন শাখার কাজকর্ম নিয়ে অবশ্য ক্ষোভপ্রকাশ করেছেন রাজীব কুমার। তাঁর বক্তব্য, অনেক ঘটনায় অভিযুক্তদের গ্রেফতার করা হলেও অপরাধ কমছে না। কমিশনারের বক্তব্য, ওই ঘটনাপ্রবাহ বলছে, একটি দল ধরা পড়লেও সমান্তরালভাবে দ্বিতীয় একটি দল তৈরি হয়ে যাচ্ছে। অর্থাৎ, ওই দুই বিভাগের ‘সোর্স নেটওয়ার্ক’ ঠিকমতো কাজ করছে না। ঘটনাচক্রে, ফেব্রুয়ারির ৩ তারিখে রাজীব কুমার দায়িত্ব নেওয়ার পরপরই শহরে কয়েকটি ডাকাতির ঘটনা ঘটেছে। তবে বৈঠকে এন্টালির গুলিচালনা নিয়ে কমিশনার তেমনকিছু বলেছেন বলে খবর নেই।\xa0\nপুলিশ বাহিনীতে রাজীব কুমার দক্ষ অফিসার বলেই পরিচিত। কিন্তু গত কয়েক বছরে ‘দক্ষতা’ ছাপিয়ে তাঁর ‘শাসক-আনুগত্য’ নিয়েই বেশি আলোচনা হয়েছে। বিশেষত, সারদা-কেলেঙ্কারির তদন্তে তাঁর ভূমিকা নিয়ে বারংবার প্রশ্ন তুলেছেন বিরোধীরা। প্রকাশ্যেই তাঁরা বলেছেন, ওই তদন্তে রাজ্য সরকারের হয়ে ‘বিশেষ ভূমিকা’ পালনের কারণেই মুখ্যমন্ত্রী মমতা বন্দ্যোপাধ্যায়ের আস্থা অর্জন করেছেন রাজীব কুমার। এবং সে কারণেই তাঁকে কলকাতার পুলিশ কমিশনার করা হয়েছে।\xa0\nএদিনের ক্রাইম কনফারেন্স অবশ্য বলছে, ‘বার্তা’ মানলে বাহিনীর সঙ্গে শাসকের সংঘাত অনিবার্য। কুমারের পক্ষে তা সম্ভব?'

In [40]:
c='''যশপ্রীত বুমরা, ভুবনেশ্বর কুমার, মহম্মদ শামি সম্বলিত ভারতীয় বোলিং আক্রমণকে বিশ্বের অন্যতম সেরা বললেন ভেঙ্কটেশ প্রসাদ।

জাভাগল শ্রীনাথের সঙ্গে তাঁর বল হাতে পার্টনারশিপ নিয়ে এখনও অনেক কথা হয় ভারতীয় ক্রিকেটে। স্লোয়ার দেওয়ার জন্য বিখ্যাত প্রসাদ বলছেন, “এই মুহূর্তে আমাদের ফাস্ট বোলিং বিশ্বসেরা। বিরাটের নেতৃত্বে ভারতের বোলিং বিভাগ দারুণ শক্তিশালী, প্রতিভাসম্পন্ন।”

প্রসাদ ও শ্রীনাথ ক্রিকেটকে বিদায় জানানোর পরে ভারতীয় ফাস্ট বোলিংকে কিছুকাল নেতৃত্ব দেন জাহির খান। চোট তাঁর কেরিয়ারকে দীর্ঘায়িত করতে দেয়নি। প্রসাদ বলছেন, “জাহির খান সফল বোলার। কিন্তু জাহির ভাল পার্টনার পায়নি। কেউ এসেছে আবার কেউ চলে গিয়েছে দ্রুত। ফলে পার্টনারশিপ দানা বাঁধেনি। আশিস নেহরা অল্প কয়েক দিন ছিল। অজিত আগরকরও তাই। প্রবীণ কুমার, আরপি সিংহরাও বেশি দিন স্থায়ী হয়নি।”'''

In [43]:
b='''বুধবার সমস্ত রাজ্য ক্রিকেট সংস্থাগুলোর কাছে পাঠানো চিঠিতে সৌরভ লিখেছেন, "এ বছরই আইপিএল আয়োজন করার আমরা সব রকম চেষ্টা করছি। দর্শকশূন্য মাঠেও যদি খেলা আয়োজন করতে হয়, সেটাই করা হবে। ক্রিকেটপ্রেমী, ফ্র্যাঞ্চাইজি, প্লেয়ার-সহ বাকিরা তাকিয়ে রয়েছে আইপিএল-এর দিকে। বিসিসিআই এ ব্যাপারে দ্রুত সিদ্ধান্ত নেবে।"

সম্প্রতি দেশ-বিদেশের ক্রিকেটাররা এ বারের আইপিএল-এ খেলার ব্যাপারে আগ্রহ প্রকাশ করেছেন। তাঁরাও চান, চলতি বছরেই আইপিএল হোক। সৌরভের বোর্ডও সে দিকেই নজর দিচ্ছে। বোর্ড প্রেসিডেন্টের চিঠিতেই তা স্পষ্ট। যদিও আইপিএল হওয়া, না হওয়া ঝুলে রয়েছে টি টোয়েন্টি বিশ্বকাপের উপরে।

সব কিছু ঠিকঠাক থাকলে ১৮ অক্টোবর থেকে শুরু হওয়ার কথা ছিল বিশ্বকাপ। করোনাভাইরাস নিয়ে আতঙ্কের মধ্যে কি অস্ট্রেলিয়ায় আদৌ বিশ্বকাপ করা সম্ভব? অস্ট্রেলিয়া ক্রিকেট বোর্ড এখনও এ ব্যাপারে খোলাখুলি কিছু জানাতে পারেনি। অনেকেই মনে করছেন এই পরিস্থিতিতে অস্ট্রেলিয়ায় টি টোয়েন্টি বিশ্বকাপ আয়োজন করা সম্ভব হবে না।

আইসিসি সময় নিচ্ছে। আর আইসিসি সময় নেওয়ায় ভারতীয় বোর্ডও আইপিএল নিয়ে সিদ্ধান্ত নিতে পারছে না। আইপিএল না হলে যেমন প্রচুর আর্থিক লোকসান রয়েছে, তেমনই রুদ্ধদ্বার স্টেডিয়ামে টুর্নামেন্ট হলে তা রং হারাবে, এ কথা বলাই বাহুল্য।

বোর্ড্ প্রেসিডেন্ট সৌরভের সামনে কঠিন চ্যালেঞ্জ। ক্যাপ্টেন থাকার সময়ে বহু কঠিন ম্যাচ সৌরভ বের করে এনেছেন বিচক্ষণ সিদ্ধান্ত নিয়ে। এ বারও বোর্ড প্রেসিডেন্ট কিছু একটা উপায় বের করবেন বলেই মনে করছেন ক্রিকেট বিশেষজ্ঞরা।'''

In [41]:
c

'যশপ্রীত বুমরা, ভুবনেশ্বর কুমার, মহম্মদ শামি সম্বলিত ভারতীয় বোলিং আক্রমণকে বিশ্বের অন্যতম সেরা বললেন ভেঙ্কটেশ প্রসাদ।\n\nজাভাগল শ্রীনাথের সঙ্গে তাঁর বল হাতে পার্টনারশিপ নিয়ে এখনও অনেক কথা হয় ভারতীয় ক্রিকেটে। স্লোয়ার দেওয়ার জন্য বিখ্যাত প্রসাদ বলছেন, “এই মুহূর্তে আমাদের ফাস্ট বোলিং বিশ্বসেরা। বিরাটের নেতৃত্বে ভারতের বোলিং বিভাগ দারুণ শক্তিশালী, প্রতিভাসম্পন্ন।”\n\nপ্রসাদ ও শ্রীনাথ ক্রিকেটকে বিদায় জানানোর পরে ভারতীয় ফাস্ট বোলিংকে কিছুকাল নেতৃত্ব দেন জাহির খান। চোট তাঁর কেরিয়ারকে দীর্ঘায়িত করতে দেয়নি। প্রসাদ বলছেন, “জাহির খান সফল বোলার। কিন্তু জাহির ভাল পার্টনার পায়নি। কেউ এসেছে আবার কেউ চলে গিয়েছে দ্রুত। ফলে পার্টনারশিপ দানা বাঁধেনি। আশিস নেহরা অল্প কয়েক দিন ছিল। অজিত আগরকরও তাই। প্রবীণ কুমার, আরপি সিংহরাও বেশি দিন স্থায়ী হয়নি।”'

In [52]:
d='''এই অতিমারির কবলে দেশবাসীর প্রতি নানা ভাবে সাহায্যের হাত বাড়িয়ে দিচ্ছেন বলিউডের বিভিন্ন তারকারা। ক্যাটরিনা কাইফও দিনমজুরদের পাশে দাঁড়িয়েছেন। একটি সংস্থার সঙ্গে যুক্ত হয়ে তাঁদের কাছে খাবার ও প্রয়োজনীয় সামগ্রী পৌঁছে দিচ্ছেন। তাঁদের কাজের একটি ভিডিয়ো পোস্ট করে তিনি দেশবাসীকে আহ্বান জানিয়েছেন দুর্গতদের সাহায্যার্থে এগিয়ে আসার জন্য। ক্যাটরিনার কথায়, এই মুহূর্তে প্রত্যেকের সহযোগিতা প্রয়োজন। 

অন্য দিকে ৪০ জন ব্যাকআপ ডান্সারের পাশে দাঁড়িয়েছেন শাহিদ কপূর। তাঁর গোড়ার দিকের ছবি ‘ইশক ভিশক’ থেকে এখনও পর্যন্ত যত ছবি করেছেন, সেই ছবির ব্যাকআপ ডান্সারদের থেকে শর্টলিস্ট করে এই ৪০ জনকে বেছে নেওয়া হয়েছে। এঁদের প্রত্যেকের অ্যাকাউন্টে এ মাসে টাকা ট্রান্সফার করেছেন শাহিদ। আগামী দু’-তিন মাসও এই ৪০ জনের অ্যাকাউন্টে তিনি টাকা পাঠাবেন। এক সময়ে ‘তাল’-এর মতো ছবিতে শাহিদ নিজেও ব্যাকআপ ডান্সারের ভূমিকায় ছিলেন। 

পরিযায়ী শ্রমিকদের বাড়ি ফিরতে সাহায্য করছেন অমিতাভ বচ্চন। মুম্বই থেকে তিনটি ফ্লাইট বুক করে বারাণসীতে ৫০০ জন শ্রমিককে বাড়ি ফেরার ব্যবস্থা করে দিয়েছেন তিনি। এর আগেও অমিতাভ অনেক পরিযায়ী শ্রমিককে উত্তরপ্রদেশে যাওয়ার জন্য বাসের ব্যবস্থা করে দিয়েছিলেন। এ বার ট্রেনেই বারাণসী যাওয়ার ব্যবস্থা করার কথা ছিল। কিন্তু সে কাজে বাধা সৃষ্টি হলে বিমানে ফেরার ব্যবস্থা করে দেন। শোনা যাচ্ছে, পশ্চিমবঙ্গ, বিহার ও তামিলনাড়ুর পরিযায়ী শ্রমিকদেরও বাড়িতে ফেরার ব্যবস্থা করবেন খুব শিগগির। '''

In [53]:
print(text_cl_svm.predict([d]))

['entertainment']


In [55]:
df=pd.read_csv('../UPDATED_NLP_COURSE/TextFiles/amazonreviews.tsv',sep='\t')
df.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [57]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

In [58]:
from sklearn.model_selection import train_test_split

In [62]:
x_train,x_test,y_train,y_test=train_test_split(df.review,df.label,test_size=0.2)

In [60]:
text_cl_nb=Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())])
text_cl_svm=Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [63]:
text_cl_nb.fit(x_train,y_train)
text_cl_svm.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [64]:
pred_nb=text_cl_nb.predict(x_test)
pred_svm=text_cl_svm.predict(x_test)

In [65]:
print(accuracy_score(y_test,pred_nb))
print(accuracy_score(y_test,pred_svm))

0.8285
0.8665


In [66]:
df.head(10)

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."
5,pos,an absolute masterpiece: I am quite sure any o...
6,neg,"Buyer beware: This is a self-published book, a..."
7,pos,Glorious story: I loved Whisper of the wicked ...
8,pos,A FIVE STAR BOOK: I just finished reading Whis...
9,pos,Whispers of the Wicked Saints: This was a easy...


In [67]:
text='Excellent movie. I suggest everyone to see the movie and enjoy'
text1='Too Bad cinema. Actor was good but story line was worst. Do not waste time'

In [70]:
print(text_cl_svm.predict([text1]))

['neg']
